<a href="https://colab.research.google.com/github/dalejo3000/pygraphs/blob/main/Dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install dash

import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

# Cargar el dataset
file_path = './clinical_analytics.csv'  # Cambia la ruta si es necesario
df = pd.read_csv(file_path)

# Preprocesamiento: Agregación previa de los datos necesarios

admit_source_summary = df.groupby('Admit Source').size().reset_index(name='count')
wait_time_summary = df.groupby('Department').agg({'Wait Time Min': 'mean'}).reset_index()
clinic_distribution = df['Clinic Name'].value_counts().reset_index()
clinic_distribution.columns = ['Clinic Name', 'count']

# Crear la aplicación Dash
app = Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("Clinical Analytics Dashboard", style={'textAlign': 'center', 'marginBottom': '20px', 'color': 'white'}),

    # Filtros
    html.Div([
        html.Label("Filtrar por Departamento:", style={'fontWeight': 'bold', 'color': 'white'}),
        dcc.Dropdown(
            id='filter-department',
            options=[{'label': dept, 'value': dept} for dept in df['Department'].dropna().unique()],
            placeholder="Selecciona uno o más Departamentos",
            multi=True,
            style={'width': '45%', 'marginBottom': '10px'}
        ),
        html.Label("Filtrar por Admit Source:", style={'fontWeight': 'bold', 'color': 'white'}),
        dcc.Dropdown(
            id='filter-admit-source',
            options=[{'label': source, 'value': source} for source in df['Admit Source'].dropna().unique()],
            placeholder="Selecciona un Admit Source",
            style={'width': '45%', 'marginBottom': '10px'}
        )
    ], style={'width': '60%', 'margin': 'auto', 'textAlign': 'center'}),

    # Gráficos en cuadrícula
    html.Div([
        html.Div(dcc.Graph(id='bar-chart', style={'height': '350px'}), style={'flex': '1', 'padding': '10px'}),
        html.Div(dcc.Graph(id='scatter-chart', style={'height': '350px'}), style={'flex': '1', 'padding': '10px'})
    ], style={'display': 'flex'}),

    html.Div([
        html.Div(dcc.Graph(id='pie-chart', style={'height': '350px'}), style={'flex': '1', 'padding': '10px'}),
        html.Div(dcc.Graph(id='bar-chart-department', style={'height': '350px'}), style={'flex': '1', 'padding': '10px'})
    ], style={'display': 'flex'}),
], style={'backgroundColor': 'black', 'padding': '20px'})


@app.callback(
    [Output('bar-chart', 'figure'),
     Output('scatter-chart', 'figure'),
     Output('pie-chart', 'figure'),
     Output('bar-chart-department', 'figure')],
    [Input('filter-department', 'value'),
     Input('filter-admit-source', 'value')]
)
def update_charts(departments, admit_source):
    filtered_wait_time = wait_time_summary.copy()
    filtered_admit_source = admit_source_summary.copy()
    filtered_bar_data = df.groupby(['Department']).size().reset_index(name='count')

    if departments:
        filtered_wait_time = filtered_wait_time[filtered_wait_time['Department'].isin(departments)]
        filtered_bar_data = filtered_bar_data[filtered_bar_data['Department'].isin(departments)]

    if admit_source:
        filtered_admit_source = filtered_admit_source[filtered_admit_source['Admit Source'] == admit_source]

    # Gráfico 1: Admisiones por Admit Source
    bar_chart = px.bar(
        filtered_admit_source,
        x='Admit Source',
        y='count',
        title='Distribución por Admit Source',
        template='plotly_white',
        color_discrete_sequence=px.colors.qualitative.Dark24
    )
    bar_chart.update_layout(margin=dict(l=20, r=20, t=40, b=20), height=350)

    # Gráfico 2: Tiempo de espera promedio por Departamento (Scatter Plot)
    scatter_chart = px.scatter(
        filtered_wait_time,
        x='Department',
        y='Wait Time Min',
        title='Tiempo de Espera Promedio por Departamento (Scatter Plot)',
        template='plotly_white',
        color='Department',
        size='Wait Time Min',
        color_discrete_sequence=px.colors.qualitative.Alphabet
    )
    scatter_chart.update_layout(
        margin=dict(l=20, r=20, t=40, b=20),
        height=350,
        xaxis_title='Departamento',
        yaxis_title='Tiempo de Espera (Min)'
    )

    # Gráfico 3: Distribución por Clínica u Hospital
    pie_chart = px.pie(
        clinic_distribution,
        names='Clinic Name',
        values='count',
        title='Distribución por Clínica u Hospital',
        template='plotly_white',
        color_discrete_sequence=px.colors.qualitative.Alphabet
    )
    pie_chart.update_layout(margin=dict(l=20, r=20, t=40, b=20), height=350)

    # Gráfico 4: Gráfico de Barras por Departamento
    bar_chart_department = px.bar(
        filtered_bar_data,
        x='Department',
        y='count',
        title='Cantidad de Registros por Departamento',
        template='plotly_white',
        color='Department',
        color_discrete_sequence=px.colors.qualitative.Pastel2
    )
    bar_chart_department.update_layout(
        xaxis_title='Departamento',
        yaxis_title='Cantidad de Registros',
        margin=dict(l=20, r=20, t=40, b=20),
        height=350
    )

    return bar_chart, scatter_chart, pie_chart, bar_chart_department


# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True, port=8050)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


<IPython.core.display.Javascript object>